Aim - Bayesian Hyperparameter Optimization using scikit-learn

1.Scikit-Optimize
2.Machine Learning Dataset and Model
3.Manually Tune Algorithm Hyperparameters
4.Automatically Tune Algorithm Hyperparameters

In [27]:
import skopt
print('skopt %s' % skopt.__version__)

skopt 0.10.1


In [28]:
# summarize the ionosphere dataset
from pandas import read_csv
# load dataset
data_path = 'C:\\Users\\obero\\Desktop\\RUTGERS\\Projects\\Bayesian-Hyperparameter-Optimization\\data\\ionosphere.data'
dataframe = read_csv(data_path, header=None)
# split into input and output elements
data = dataframe.values
X, y = data[:, :-1], data[:, -1]
print(X.shape, y.shape)

(351, 34) (351,)


In [29]:

# evaluate an svm for the ionosphere dataset
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
# split into input and output elements
data = dataframe.values
X, y = data[:, :-1], data[:, -1]
print(X.shape, y.shape)
# define model model
model = SVC()
# define test harness
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
m_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy: %.3f (%.3f)' % (mean(m_scores), std(m_scores)))

(351, 34) (351,)
Accuracy: 0.937 (0.038)


We will tune the following hyperparameters of the SVM model:

C, the regularization parameter.

kernel, the type of kernel used in the model.

degree, used for the polynomial kernel.

gamma, used in most other kernels.

For the numeric hyperparameters C and gamma, we will define a log scale to search between a small value of 1e-6 and 100. Degree is an integer and we will search values between 1 and 5. Finally, the kernel is a categorical variable with specific named values

In [30]:
from skopt.space import Real, Categorical, Integer
# define the space of hyperparameters to search
search_space = list()
search_space.append(Real(1e-6, 100.0, 'log-uniform', name='C'))
search_space.append(Categorical(['linear', 'poly', 'rbf', 'sigmoid'], name='kernel'))
search_space.append(Integer(1, 5, name='degree'))
search_space.append(Real(1e-6, 100.0, 'log-uniform', name='gamma'))

In [31]:
from skopt.utils import use_named_args
# define the function used to evaluate a given configuration
@use_named_args(search_space)
def evaluate_model(**params):
	# configure the model with specific hyperparameters
	model = SVC()
	model.set_params(**params)
	# define test harness
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	# calculate 5-fold cross validation
	result = cross_val_score(model, X, y, cv=cv, n_jobs=-1, scoring='accuracy')
	# calculate the mean of the scores
	estimate = mean(result)
	# convert from a maximizing score to a minimizing score
	return 1.0 - estimate

In [32]:

# perform optimization
result = skopt.gp_minimize(evaluate_model, search_space)

In [33]:
...
# summarizing finding:
print('Best Accuracy: %.3f' % (1.0 - result.fun))
print('Best Parameters: %s' % (result.x))

Best Accuracy: 0.954
Best Parameters: [16.52508913224459, 'rbf', 1, 0.0332975050985643]


Automatically Tune Algorithm Hyperparameters

The Scikit-Learn machine learning library provides tools for tuning model hyperparameters.

Specifically, it provides the GridSearchCV and RandomizedSearchCV classes that take a model, a search space, and a cross-validation configuration.

The benefit of these classes is that the search procedure is performed automatically, requiring minimal configuration.

Similarly, the Scikit-Optimize library provides a similar interface for performing a Bayesian Optimization of model hyperparameters via the BayesSearchCV class.

This class can be used in the same way as the Scikit-Learn equivalents.

First, the search space must be defined as a dictionary with hyperparameter names used as the key and the scope of the variable as the value.

In [35]:
# define search space
params = dict()
params['C'] = (1e-6, 100.0, 'log-uniform')
params['gamma'] = (1e-6, 100.0, 'log-uniform')
params['degree'] = (1,5)
params['kernel'] = ['linear', 'poly', 'rbf', 'sigmoid']

In [36]:
# define evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define the search
search = skopt.BayesSearchCV(estimator=SVC(), search_spaces=params, n_jobs=-1, cv=cv)

In [41]:
# perform the search
search.fit(X, y)
# report the best result
print(search.best_score_)
print(search.best_params_)

0.8840476190476191
OrderedDict({'C': 100.0, 'degree': 2, 'gamma': 40.597241934366636, 'kernel': 'linear'})
